In [1]:
%matplotlib inline
import matplotlib
matplotlib.rc('font', **{'size': 22})

In [2]:
import matplotlib.pyplot as plt

In [3]:
import elasticsearch
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch([{'host': 'atlas-kibana.mwt2.org',
                     'port': 9200}])

In [4]:
body_epochs="""{
  "size": 500,
  "sort": [
    {
      "@timestamp": {
        "order": "desc",
        "unmapped_type": "boolean"
      }
    }
  ],
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "analyze_wildcard": true,
          "lowercase_expanded_terms": false,
          "query": "payload.activity:Production Input"
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "range": {
                "@timestamp": {
                  "gte": 1467079481752,
                  "lte": 1467122681752,
                  "format": "epoch_millis"
                }
              }
            }
          ],
          "must_not": []
        }
      }
    }
  },
  "highlight": {
    "pre_tags": [
      "@kibana-highlighted-field@"
    ],
    "post_tags": [
      "@/kibana-highlighted-field@"
    ],
    "fields": {
      "*": {}
    },
    "require_field_match": false,
    "fragment_size": 2147483647
  },
  "aggs": {
    "2": {
      "date_histogram": {
        "field": "@timestamp",
        "interval": "10m",
        "time_zone": "Europe/Berlin",
        "min_doc_count": 0,
        "extended_bounds": {
          "min": 1467079481751,
          "max": 1467122681751
        }
      }
    }
  },
  "fields": [
    "*",
    "_source"
  ],
  "script_fields": {},
  "fielddata_fields": [
    "@timestamp"
  ]
}"""

In [5]:
data = []
scanner = helpers.scan(client=es, query=body_epochs, scroll='1m',
                       index="rucio-events*", timeout="1m")
for hit in scanner:
    data.append((hit['_source']['payload']['file-size']/1000))

RequestError: TransportError(400, u'search_phase_execution_exception')